In [ ]:
# @title
# Software setup
!pip install autogen~=0.2
import random
import numpy as np
import matplotlib.pyplot as plt
from autogen import ConversableAgent, UserProxyAgent
llm_config = {
    "model": "gpt-5-mini",
    "api_key":  "",
    "base_url": "https://api.openai.com/v1"
}

In [ ]:
# Truths about the world
# Topic names
date_topic = {
    "September 30":
    "Individual Foundations / Agents",
    "October 7":
    "Homophily",
    "October 14":
    "Simple/complex Contagion and Networks",
    "October 21":
    "Belief Change: Can LLMs Model Belief Change and Persuade It to Occur?",
    "October 28":
    "Cooperation and Coordination: Norms and Conventions",
    "November 11":
    "Social (Imitation) and Organizational Learning Processes"
    }
# Number of words student wrote before 1pm each week on Canvas
date_outcome = {
    "September 30": 2217,
    "October 7": 5124,
    "October 14": 2808,
    "October 21": 2582,
    "October 28": 1950,
    "November 11": 1562
    }

In [ ]:
# @title
# This function initializes a 1:1 conversation between the user and an agent i
def initialize(i):
  agent = ConversableAgent(f"Agent_{i+1}", llm_config = llm_config)
  agents.append(agent)
  chat = user.initiate_chat(
    agent,
    message = f"""Please help me with an estimation game.
    This fall quarter at Stanford University, Professors Dan McFarland and Amir Goldberg are teaching a seminar called Rediscovering Social Science through Multiagent Simulations.
    Sixteen students are currently registered to take the class.
    Students are asked to submit comments on each week's material by 1pm on Tuesday, 2 hours before class.
    Now, please estimate the total number of words students wrote for class by 1pm on Tuesday, {dates[0]}.
    The topic of the week was {date_topic[dates[0]]}.
    Please answer with just the estimated number of words."""
    )
  chats.append(chat)

# This function saves the predictions made by agent i along with an explanation
def get_preds_for_agent(i):
  # global preds
  # global explanations
  preds.append(float(chats[i].chat_history[history_counter]["content"]))
  user.send("Please explain how you came to this estimate.", agents[i])
  explanations.append(chats[i].chat_history[history_counter + 2]["content"])

# This function exposes agent i to the predictions and explanations from j
# other agents before asking for a new prediction
def make_interact(i):
  others = list(range(n_agents))
  del others[i]
  others = random.sample(others, n_others)
  others_message = ""
  for j in others:
    others_message = others_message + f"""Another person, with ID number {j + 1}, predicted {preds_before_interaction[dates[date]][j]}."""
    if explanation:
      others_message = others_message + f"""Here's their explanation: {explanations_before_interaction[dates[date]][j]}"""
  others_message = others_message + f"""Now, again, please estimate the total number of words the students wrote for class by 1pm on Tuesday, {dates[date]}.
  Please answer with just the estimated number of words."""
  user.send(others_message, agents[i])
  preds.append(float(chats[i].chat_history[history_counter + 4]["content"]))

# This function gets predictions and explanations from i agents and, if
# necessary, obtains a new set of predictions after exposing each agent to j
# other agents' predictions and reasonings
def get_preds():
  # global chats
  global preds
  global explanations
  global history_counter
  preds = []
  explanations = []
  for i in range(n_agents):
    get_preds_for_agent(i)
  preds_before_interaction.update({dates[date]: preds})
  if rep == 0:
    preds_before_interaction_avg.update({dates[date]: []})
    preds_before_interaction_std.update({dates[date]: []})
    preds_before_interaction_mae.update({dates[date]: []})
    preds_before_interaction_msd.update({dates[date]: []})
  avg = sum(preds) / n_agents
  std = np.std(preds, ddof = 1)
  errors = [pred - date_outcome[dates[date]] for pred in preds]
  mae = sum([abs(x) for x in errors]) / n_agents
  msd = abs(sum(preds)/ n_agents - date_outcome[dates[date]])
  preds_before_interaction_avg[dates[date]].append(avg)
  preds_before_interaction_std[dates[date]].append(std)
  preds_before_interaction_mae[dates[date]].append(mae)
  preds_before_interaction_msd[dates[date]].append(msd)
  explanations_before_interaction.update({dates[date]: explanations})
  if n_others == 0:
    history_counter = history_counter + 2
  else:
    preds = []
    for i in range(n_agents):
      make_interact(i)
    preds_after_interaction.update({dates[date]: preds})
    if rep == 0:
      preds_after_interaction_avg.update({dates[date]: []})
      preds_after_interaction_std.update({dates[date]: []})
      preds_after_interaction_mae.update({dates[date]: []})
      preds_after_interaction_msd.update({dates[date]: []})
    avg = sum(preds) / n_agents
    std = np.std(preds, ddof = 1)
    errors = [pred - date_outcome[dates[date]] for pred in preds]
    mae = sum([abs(x) for x in errors]) / n_agents
    msd = abs(sum(preds)/ n_agents - date_outcome[dates[date]])
    preds_after_interaction_avg[dates[date]].append(avg)
    preds_after_interaction_std[dates[date]].append(std)
    preds_after_interaction_mae[dates[date]].append(mae)
    preds_after_interaction_msd[dates[date]].append(msd)
    history_counter = history_counter + 4

# This function tells agent i what the correct answer is before asking them
# to generate a new set of predictions for the next date.
def predict_next_for_agent(i):
  user.send(f"""The actual number of words students wrote was {date_outcome[dates[date]]}.
  Now, please estimate the total number of words students wrote for class by 1pm on Tuesday, {dates[date + 1]}.
  The topic of the week was {date_topic[dates[date + 1]]}.
  Please answer with just the estimated number of words.""", agents[i])

# This function tells i agents what the correct answer is before asking them
# to generate a new set of predictions for the next date. One can get those
# predictions using get_preds()
def predict_next():
  global date
  global history_counter
  for i in range(n_agents):
    predict_next_for_agent(i)
  date = date + 1
  history_counter = history_counter + 2

def run_job(n_agents, n_others, dates, rep):
  # Set up empty holder variables and counter variables
  global agents
  global chats
  global user
  global preds_before_interaction
  global explanations_before_interaction
  global preds_after_interaction
  global date
  global history_counter
  agents = []
  chats = []
  preds_before_interaction = {}
  explanations_before_interaction = {}
  preds_after_interaction = {}
  date = 0
  history_counter = 1

  # Initialize a user proxy agent to talk to conversable agents
  user = UserProxyAgent(
      "User",
      max_consecutive_auto_reply = 0,
      human_input_mode = "NEVER"
      )

  # Initiatialize conversations and get the first round of predictions (or
  # first 2 rounds if n_agents > 0)
  for i in range(n_agents):
    initialize(i)
  get_preds()

  # Move on to predicting the next date
  for i in range(len(dates) - 1):
    predict_next()
    get_preds()



In [ ]:
# Job setting
# How many forecasters?
n_agents = 5

# For which days will the forecasters make predictions? In future applications,
# these do not have to be dates. For example, they can be userids.
dates = ["October 21", "October 28"]

# How many other random forecasters' predictions and reasonings will each
# forecaster see for each date
n_others = 1 # 1

# Will forecasters see each others' explanations
explanation = False

# How many times do you want to re-run the entire simulation?
n_reps = 16

# This is where the deliverables are going to be saved.
# Each of the following lines will be a dictionary with n entries where
# each entry corresponds to a date
# avg refers to the average prediction across agents
# std refers to the standard deviation of predictions
# mae refers to the mean absolute error across agents
# msd refers to the mean signed deviation (0 if the errors cancel out)
# before refers to predictions before any social interaction for each date
# after refers to predictions after any social interaction for each date
preds_before_interaction_avg = {}
preds_before_interaction_mae = {}
preds_before_interaction_msd = {}
preds_before_interaction_std = {}
preds_after_interaction_avg = {}
preds_after_interaction_mae = {}
preds_after_interaction_msd = {}
preds_after_interaction_std = {}

In [ ]:
# Let's simulate!
for rep in range(n_reps):
  run_job(n_agents, n_others, dates, rep)
# Just run again if you encounter some rare bug (e.g. the LLM outputting a
# number with a comma, happens maybe 1 in 100 times)

In [ ]:
# If you are in conditions where n_others = 0, enter the LAST entry from each
# of the following into the Google Sheet. You should see only 1 entry if you
# have "October 28" and 2 entries if you have ["October 21", "October 28"]
print(preds_before_interaction_avg)
print(preds_before_interaction_std)
print(preds_before_interaction_mae)
print(preds_before_interaction_msd)

In [ ]:
# If you are in conditions where n_others = 1, enter the LAST entry from each
# of the following into the Google Sheet. You should see only 1 entry if you
# have "October 28" and 2 entries if you have ["October 21", "October 28"]
print(preds_after_interaction_avg)
print(preds_after_interaction_std)
print(preds_after_interaction_mae)
print(preds_after_interaction_msd)